In [5]:
# conda activate bio 
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import re
import os 
import json
from tqdm import tqdm
from biopandas.pdb import PandasPdb

In [6]:
df = pd.read_parquet("../bldb/clean/csv/bldb_dset.parquet")
df

,#name,seq,length,filename,bla_class,protein_name,protein_family_filename,protein_family,seq_id,ambler_class,...,gravy,isoelectric_point,entropy,helix,turn,sheet,pass_the_filter,is_clust_rep_30,is_clust_rep_60,is_clust_rep_90
0,QOI11476.1|AAK-1| class A beta-lactamase AAK-1,MRYIRLCIISLFATLPLAVHASPQPLQQITLRESQLSGRVGTIEMD...,286,A-AAK-1-prot.fasta,class_a,AAK-1,AAK,AAK,bldb_000001,A,...,-0.025874,9.175737,3.999562,0.293706,0.185315,0.321678,yes,no,no,yes
1,gi|5596421|emb|CAB51471.1|ACI-1| class A exten...,MKKFCFLFLIICGLMVFCLQDCQARQKLNLADLENKYNAVIGVYAV...,284,A-ACI-1-prot.fasta,class_a,ACI-1,ACI,ACI,bldb_000002,A,...,-0.316901,8.350993,4.124073,0.295775,0.204225,0.246479,yes,no,yes,yes
2,WP_247239681.1|ACI-10| ACI family class A beta...,MKKFCFLFLIICGLMVFCPQDCQARQKLNLADLENKYNAVIGVYAV...,284,A-ACI-10-prot.fasta,class_a,ACI-10,ACI,ACI,bldb_000003,None,...,-0.317606,8.350993,4.125713,0.295775,0.207746,0.242958,yes,no,no,no
3,WP_296333831.1|ACI-11| ACI family class A beta...,MKKFCFLFLIICGLMVFCPQDCQARQKLNLADLENKYNAVIGVYAV...,284,A-ACI-11-prot.fasta,class_a,ACI-11,ACI,ACI,bldb_000004,None,...,-0.326761,8.350993,4.130316,0.292254,0.207746,0.246479,yes,no,no,no
4,WP_279133745.1|ACI-12| ACI family class A beta...,MKKFCFLFLIICGLMVFCLQDCQARQKLNLADLENKYNAVIGVYAV...,284,A-ACI-12-prot.fasta,class_a,ACI-12,ACI,ACI,bldb_000005,None,...,-0.289789,8.350026,4.109998,0.299296,0.204225,0.246479,yes,no,no,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29391,WP_019923896.1|XD-XD1-1| Duganella zoogloeoide...,MMPIDLDIVLARLLLAAAGSLAAGGAVWAVAVLCRRTLPALAQQRS...,631,XD-XD1-1-prot.fasta,other,XD-XD1-1,Unknown,Unknown,bldb_029441,None,...,0.138352,9.808043,3.923103,0.302694,0.206022,0.366086,no,no,no,no
29392,KQN73069.1|XD-XD1-2| Duganella sp. Leaf61,MMPIDLDIVLARLLLAAAGSLAAGGAVWAVAVLCRRTLPALAQQRS...,631,XD-XD1-2-prot.fasta,other,XD-XD1-2,Unknown,Unknown,bldb_029442,None,...,0.152932,9.827383,3.932876,0.302694,0.204437,0.359746,no,no,no,no
29393,OFA09008.1|XD-XD1-3| Duganella phyllosphaerae,MMPIDLDVVLARLLLAAAGSLAAGGAVWAVAVLCRRTLPALAQQRS...,632,XD-XD1-3-prot.fasta,other,XD-XD1-3,Unknown,Unknown,bldb_029443,None,...,0.133386,9.592589,3.925437,0.306962,0.200949,0.363924,no,no,no,no
29394,SHM90367.1|XD-XD2-1| Duganella sacchari,MTGFDIALVRLLLAAAGSLAAGGAVWGVALLCRRYLPALAQHRSLW...,597,XD-XD2-1-prot.fasta,other,XD-XD2-1,Unknown,Unknown,bldb_029444,None,...,0.094305,9.327109,3.936714,0.318258,0.197655,0.351759,no,no,no,no


In [7]:
# selec cols 
selected_cols =  [
    "Entry","Entry Name","Protein names","Gene Names","Organism","Taxonomic lineage",
    "Sequence","EC number","Signal peptide","PubMed ID","DOI ID","RefSeq",
    "AlphaFoldDB","Pfam","PDB"]

# read the data from uniprot
pbp   = pd.read_parquet("../uniprot/raw/pbp/csv/CL0013.parquet", columns=selected_cols)
metal = pd.read_parquet("../uniprot/raw/metal/csv/CL0381.parquet", columns=selected_cols)

# filter only the rows with AF2 models
pbp   = pbp[pbp.AlphaFoldDB.notnull()]
metal = metal[metal.AlphaFoldDB.notnull()]

# find seqs that the unport dset has in common to bldb
pbp   = pbp[pbp["Sequence"].isin(df["seq"])]
metal = metal[metal["Sequence"].isin(df["seq"])]

#concat dsets
df_af2 = pd.concat([pbp, metal], ignore_index=True)
df_af2

,Entry,Entry Name,Protein names,Gene Names,Organism,Taxonomic lineage,Sequence,EC number,Signal peptide,PubMed ID,DOI ID,RefSeq,AlphaFoldDB,Pfam,PDB
0,A0A009H9I3,A0A009H9I3_9GAMM,Beta-lactamase (EC 3.5.2.6),ampC J507_1365,Acinetobacter sp. 1295259,"cellular organisms (no rank), Bacteria (superk...",MRFKKISCLLLPPLFIFSTSIYAGNTPKDREIKKLVDQNFKPLLDK...,3.5.2.6,"SIGNAL 1..23; /evidence=""ECO:0000256|SAM:SignalP""",nan,None,WP_017386568.1;,A0A009H9I3;,PF00144;,None
1,A0A009I9X2,A0A009I9X2_ACIB9,Beta-lactamase (EC 3.5.2.6),blaCMY-10 J512_0592,Acinetobacter baumannii (strain 1295743),"cellular organisms (no rank), Bacteria (superk...",MRFKKISYLLLPSLFIFNTSIYAGNTSKDQEIKQLIDQNFKPLLEK...,3.5.2.6,"SIGNAL 1..23; /evidence=""ECO:0000256|SAM:SignalP""",nan,None,WP_032050704.1;,A0A009I9X2;,PF00144;,None
2,A0A009MI74,A0A009MI74_9GAMM,Beta-lactamase (EC 3.5.2.6),J551_2290,Acinetobacter sp. 1475718,"cellular organisms (no rank), Bacteria (superk...",MRFKKISCLLLPPLFIFSTSIYAGNTPKEQEVKKLVDQNFKPLLDK...,3.5.2.6,"SIGNAL 1..23; /evidence=""ECO:0000256|SAM:SignalP""",nan,None,WP_002118772.1;,A0A009MI74;,PF00144;,None
3,A0A009PJF4,A0A009PJF4_ACIBA,Beta-lactamase (EC 3.5.2.6),ampC J506_0647,Acinetobacter baumannii 625974,"cellular organisms (no rank), Bacteria (superk...",MRFKKISCLLLSPLFIFSTSIYAGNTPKDQEIKKLVDQNFKPLLEK...,3.5.2.6,"SIGNAL 1..23; /evidence=""ECO:0000256|SAM:SignalP""",nan,None,WP_032058755.1;,A0A009PJF4;,PF00144;,None
4,A0A009RN79,A0A009RN79_9GAMM,Beta-lactamase (EC 3.5.2.6),J536_1504,Acinetobacter sp. 809848,"cellular organisms (no rank), Bacteria (superk...",MRFKKISCLLLPPLFIFSTSIYAGNTPKDQEIKKLVDQNFKPLLDK...,3.5.2.6,"SIGNAL 1..23; /evidence=""ECO:0000256|SAM:SignalP""",nan,None,WP_032055358.1;,A0A009RN79;,PF00144;,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15662,X5JRU4,X5JRU4_ENTCL,Metallo-beta-lactamase type 2 (EC 3.5.2.6) (B2...,blaVIM-40,Enterobacter cloacae,"cellular organisms (no rank), Bacteria (superk...",MLKVISSLLVYMTASVMAVASPLAHSGEPSGEYPTVNEIPVGEVRL...,3.5.2.6,"SIGNAL 1..20; /evidence=""ECO:0000256|SAM:SignalP""",nan,None,WP_063865190.1;,X5JRU4;,PF00753;,None
15663,A0A2U8UYM6,A0A2U8UYM6_9BACT,Metallo-beta-lactamase PNGM-1 (EC 3.5.2.6),None,uncultured bacterium,"cellular organisms (no rank), Bacteria (superk...",MAGGKVTSSTGIAPKRYVYYPGSEELGPDEIRVIACGTGMPTARRA...,3.5.2.6,None,32664695,10.3390/ijms21144926,None,A0A2U8UYM6;,PF12706;,7BYQ;7BZ1;7BZ3;7BZ4;7BZI;7WI1;
15664,A0A0A7K5H4,A0A0A7K5H4_9FLAO,Metallo-beta-lactamase type 2 (EC 3.5.2.6) (B2...,M666_08355,Cellulophaga baltica 18,"cellular organisms (no rank), Bacteria (superk...",MKNTRLVILSLLLLFITGCKSPKNTVAYSSDTLKLIPLSNNAFQHI...,3.5.2.6,"SIGNAL 1..21; /evidence=""ECO:0000256|SAM:SignalP""",24428166,10.1111/1462-2920.12391,WP_029447057.1;,A0A0A7K5H4;,PF19583;,None
15665,A0A1G7CSI2,A0A1G7CSI2_9FLAO,Metallo-beta-lactamase type 2 (EC 3.5.2.6) (B2...,SAMN04487992_10177,Cellulophaga baltica,"cellular organisms (no rank), Bacteria (superk...",MKNTRLVILSLLLLFITGCKSPKNTVAYSSDTLKLIPLSNNAFQHI...,3.5.2.6,"SIGNAL 1..21; /evidence=""ECO:0000256|SAM:SignalP""",nan,None,WP_074537061.1;,A0A1G7CSI2;,PF19583;,None


In [8]:
df_af2.Entry.nunique()

15660

In [5]:
def download_json_batch(df, json_path, df_name, metrics_path, batch_size=5):
    """
    Download json files from the AFDB for a set of UniProt IDs in batches of size batch_size
      df: DataFrame with uniprot ids
      json_path: directory to save the json files
      metrics_path: directory to save the parsed json files as a single df in parquet format
      df_name: desired name to store the parquet dataset
      batch_size: size of the batch for processing
    """

    # set a progress bar
    pbar = tqdm(total=len(df), desc="Downloading json files from AFDB", leave=True)

    # create batches of UniProt IDs
    batches = [df[i:i + batch_size] for i in range(0, len(df), batch_size)]

    # create the url for api request
    for batch in batches:
        urls = [f'https://alphafold.ebi.ac.uk/api/prediction/{entry}' for entry in batch["Entry"]]
        output_files = [os.path.join(json_path, f'{entry}.json') for entry in batch["Entry"]]

        # download the json files with curl
        commands = [f'curl -s -X "GET" "{url}" -H "accept: application/json" > {output_file}' for url, output_file in zip(urls, output_files)]
        for command in commands:
            os.system(command)
            pbar.update(1)
    pbar.close()



def download_pdbs_batch(metrics_path, pdb_path, batch_size=5):
    """
    Download PDB files from the parsed df with all json files in batches of size batch_size
    """
    df = pd.read_parquet(metrics_path)

    # set a progress bar
    pbar2 = tqdm(total=len(df['pdbUrl']), desc="Downloading pdb files from AFDB", leave=True)

    # create batches of URLs
    url_batches = [df['pdbUrl'][i:i + batch_size].tolist() for i in range(0, len(df['pdbUrl']), batch_size)]

    # download PDB files in batches
    for urls in url_batches:
        commands = []
        for url in urls:
            filename = re.search(r"/files/(.+\.pdb)$", url).group(1)
            full_path = os.path.join(pdb_path, filename)
            command = f'curl -s -o "{full_path}" "{url}"'
            commands.append(command)
        
        # execute commands
        for command in commands:
            os.system(command)
            pbar2.update(1)
    pbar2.close()

def parse_json_files(json_path, df_name, metrics_path):
    data_list = []
    total_files = len([file for file in os.listdir(json_path) if file.endswith('.json')])
    
    with tqdm(total=total_files, desc='Processing JSON files') as pbar:
        for file in os.listdir(json_path):
            if file.endswith('.json'):
                filepath = os.path.join(json_path, file)
                
                try:
                    with open(filepath, 'r') as f:
                        data = json.load(f)
                except json.JSONDecodeError:
                    print(f"Error decoding JSON in {file}. Skipping.")
                    continue
                    
                    
                data_list.append({
                    "entryId":                data[0].get("entryId", ""),
                    "gene":                   data[0].get("gene", ""),
                    "uniprotAccession":       data[0].get("uniprotAccession", ""),
                    "uniprotId":              data[0].get("uniprotId", ""),
                    "uniprotDescription":     data[0].get("uniprotDescription", ""),
                    "taxId":                  data[0].get("taxId", ""),
                    "organismScientificName": data[0].get("organismScientificName", ""),
                    "uniprotStart":           data[0].get("uniprotStart", ""),
                    "uniprotEnd":             data[0].get("uniprotEnd", ""),
                    "uniprotSequence":        data[0].get("uniprotSequence", ""),
                    "modelCreatedDate":       data[0].get("modelCreatedDate", ""),
                    "latestVersion":          data[0].get("latestVersion", ""),
                    "allVersions":            data[0].get("allVersions", ""),
                    "isReviewed":             data[0].get("isReviewed", ""),
                    "isReferenceProteome":    data[0].get("isReferenceProteome", ""),
                    "cifUrl":                 data[0].get("cifUrl", ""),
                    "bcifUrl":                data[0].get("bcifUrl", ""),
                    "pdbUrl":                 data[0].get("pdbUrl", ""),
                    "paeImageUrl":            data[0].get("paeImageUrl", ""),
                    "paeDocUrl":              data[0].get("paeDocUrl", "")
                })
                pbar.update(1)

    df_metrics = pd.DataFrame(data_list)
    filename = ".".join([df_name, "parquet"])
    metrics = os.path.join(metrics_path, filename)
    df_metrics.to_parquet(metrics, index=False)
    print(f"Quality metrics saved at {metrics}")

In [7]:
download_json_batch(df=df_af2, json_path="../bldb/af2/json/", df_name="bldb_common_to_uniprot", metrics_path="../bldb/af2", batch_size=10)

In [8]:
parse_json_files(json_path="../bldb/af2/json/", df_name="bldb_common_to_uniprot", metrics_path="../bldb/af2")

Processing JSON files:  77%|███████████████████▏     | 12027/15660 [00:22<00:06, 524.21it/s]

Error decoding JSON in B1LXB8.json. Skipping.


Processing JSON files:  78%|███████████████████▌     | 12278/15660 [00:22<00:07, 476.66it/s]

Error decoding JSON in B8XTS1.json. Skipping.
Error decoding JSON in B9BP96.json. Skipping.
Error decoding JSON in B9J822.json. Skipping.
Error decoding JSON in B9KUW5.json. Skipping.
Error decoding JSON in C0LMI5.json. Skipping.
Error decoding JSON in C0M550.json. Skipping.
Error decoding JSON in C3MAK3.json. Skipping.


Processing JSON files: 100%|████████████████████████▉| 15652/15660 [00:29<00:00, 533.12it/s]


Quality metrics saved at ../bldb/af2/bldb_common_to_uniprot.parquet


In [10]:
# downloada the errors and tun again 
parse_json_files(json_path="../bldb/af2/json/", df_name="bldb_common_to_uniprot", metrics_path="../bldb/af2")

Processing JSON files: 100%|█████████████████████████| 15660/15660 [00:24<00:00, 637.97it/s]


Quality metrics saved at ../bldb/af2/bldb_common_to_uniprot.parquet


In [9]:
download_pdbs_batch(metrics_path="../bldb/af2/bldb_common_to_uniprot.parquet", pdb_path="../bldb/af2/pdbs/", batch_size=20)

In [24]:
def process_pdb_files(directory):
    plddt_lst = []
    resid_lst = []
    files_lst = []

    files = os.listdir(directory)
    with tqdm(total=len(files), desc="Processing PDB files") as pbar:
        for file in files:
            filename = os.path.join(directory, file)
            df_pdb = PandasPdb().read_pdb(filename).df['ATOM']
            df_pdb = df_pdb.groupby('residue_number')['b_factor'].mean().reset_index()

            plddt_dict = df_pdb.set_index('residue_number')['b_factor'].to_dict()
            resid_lst.append(plddt_dict)

            plddt = df_pdb.b_factor.mean()
            plddt_lst.append(plddt)

            files_lst.append(file)
            pbar.update(1)

    df_qual = pd.DataFrame({'model': files_lst, 'mean_plddt': plddt_lst, 'resid_plddt': resid_lst})
    return df_qual

In [25]:
df_plddt = process_pdb_files('../bldb/af2/pdbs/')
df_plddt

Processing PDB files: 100%|███████████████████████████| 15659/15659 [09:33<00:00, 27.33it/s]


,model,mean_plddt,resid_plddt
0,AF-A0A009H9I3-F1-model_v4.pdb,94.549060,"{1: 43.47, 2: 38.41, 3: 41.44, 4: 45.84, 5: 43..."
1,AF-A0A009HR34-F1-model_v4.pdb,92.132418,"{1: 42.47, 2: 38.53000000000001, 3: 42.97, 4: ..."
2,AF-A0A009I140-F1-model_v4.pdb,91.903540,"{1: 37.12, 2: 35.59, 3: 41.62, 4: 42.72, 5: 45..."
3,AF-A0A009I9X2-F1-model_v4.pdb,94.540470,"{1: 41.28, 2: 37.47, 3: 39.28, 4: 43.56, 5: 45..."
4,AF-A0A009MI74-F1-model_v4.pdb,94.668590,"{1: 43.53, 2: 38.19, 3: 41.25, 4: 46.75, 5: 45..."
...,...,...,...
15654,AF-X6KLF7-F1-model_v4.pdb,90.849410,"{1: 42.19, 2: 42.38, 3: 44.81, 4: 52.56, 5: 50..."
15655,AF-X7FDK5-F1-model_v4.pdb,93.604694,"{1: 37.16, 2: 39.88, 3: 39.34, 4: 41.25, 5: 42..."
15656,AF-X7R747-F1-model_v4.pdb,95.022297,"{1: 39.09, 2: 37.66, 3: 42.78, 4: 44.12, 5: 46..."
15657,AF-X7YGR2-F1-model_v4.pdb,90.442215,"{1: 35.88, 2: 37.38, 3: 39.41, 4: 41.41, 5: 42..."


In [30]:
df_plddt['resid_plddt'] = df_plddt['resid_plddt'].astype(str)
df_plddt.to_parquet("../bldb/af2/plddt_values.parquet", index=False)

# merge data

In [9]:
data = pd.read_parquet("../bldb/af2/bldb_common_to_uniprot.parquet")
data["model"] = data['pdbUrl'].apply(lambda x: re.search(r'/([^/]+)$', x).group(1))
data.head(3)

,entryId,gene,uniprotAccession,uniprotId,uniprotDescription,taxId,organismScientificName,uniprotStart,uniprotEnd,uniprotSequence,...,latestVersion,allVersions,isReviewed,isReferenceProteome,cifUrl,bcifUrl,pdbUrl,paeImageUrl,paeDocUrl,model
0,AF-A0A009H9I3-F1,ampC,A0A009H9I3,A0A009H9I3_9GAMM,Beta-lactamase,1310608,Acinetobacter sp. 1295259,1,383,MRFKKISCLLLPPLFIFSTSIYAGNTPKDREIKKLVDQNFKPLLDK...,...,4,"[3, 4]",False,True,https://alphafold.ebi.ac.uk/files/AF-A0A009H9I...,https://alphafold.ebi.ac.uk/files/AF-A0A009H9I...,https://alphafold.ebi.ac.uk/files/AF-A0A009H9I...,https://alphafold.ebi.ac.uk/files/AF-A0A009H9I...,https://alphafold.ebi.ac.uk/files/AF-A0A009H9I...,AF-A0A009H9I3-F1-model_v4.pdb
1,AF-A0A009HR34-F1,blaOXA-33,A0A009HR34,A0A009HR34_9GAMM,Beta-lactamase,1310608,Acinetobacter sp. 1295259,1,273,MTKKTLFFAIGTMFLSACSFNTVEQHQIQSISTNKNSEKIQSLFDQ...,...,4,"[3, 4]",False,True,https://alphafold.ebi.ac.uk/files/AF-A0A009HR3...,https://alphafold.ebi.ac.uk/files/AF-A0A009HR3...,https://alphafold.ebi.ac.uk/files/AF-A0A009HR3...,https://alphafold.ebi.ac.uk/files/AF-A0A009HR3...,https://alphafold.ebi.ac.uk/files/AF-A0A009HR3...,AF-A0A009HR34-F1-model_v4.pdb
2,AF-A0A009I140-F1,blaOXA-33,A0A009I140,A0A009I140_ACIB9,Beta-lactamase,1310613,Acinetobacter baumannii (strain 1295743),1,274,MNIKALLLITSAIFISACSPYIVTANPNHSASKSDEKAEKIKNLFN...,...,4,"[3, 4]",False,True,https://alphafold.ebi.ac.uk/files/AF-A0A009I14...,https://alphafold.ebi.ac.uk/files/AF-A0A009I14...,https://alphafold.ebi.ac.uk/files/AF-A0A009I14...,https://alphafold.ebi.ac.uk/files/AF-A0A009I14...,https://alphafold.ebi.ac.uk/files/AF-A0A009I14...,AF-A0A009I140-F1-model_v4.pdb


In [10]:
df_plddt = pd.read_parquet("../bldb/af2/plddt_values.parquet")
df_plddt

,model,mean_plddt,resid_plddt
0,AF-A0A009H9I3-F1-model_v4.pdb,94.549060,"{1: 43.47, 2: 38.41, 3: 41.44, 4: 45.84, 5: 43..."
1,AF-A0A009HR34-F1-model_v4.pdb,92.132418,"{1: 42.47, 2: 38.53000000000001, 3: 42.97, 4: ..."
2,AF-A0A009I140-F1-model_v4.pdb,91.903540,"{1: 37.12, 2: 35.59, 3: 41.62, 4: 42.72, 5: 45..."
3,AF-A0A009I9X2-F1-model_v4.pdb,94.540470,"{1: 41.28, 2: 37.47, 3: 39.28, 4: 43.56, 5: 45..."
4,AF-A0A009MI74-F1-model_v4.pdb,94.668590,"{1: 43.53, 2: 38.19, 3: 41.25, 4: 46.75, 5: 45..."
...,...,...,...
15654,AF-X6KLF7-F1-model_v4.pdb,90.849410,"{1: 42.19, 2: 42.38, 3: 44.81, 4: 52.56, 5: 50..."
15655,AF-X7FDK5-F1-model_v4.pdb,93.604694,"{1: 37.16, 2: 39.88, 3: 39.34, 4: 41.25, 5: 42..."
15656,AF-X7R747-F1-model_v4.pdb,95.022297,"{1: 39.09, 2: 37.66, 3: 42.78, 4: 44.12, 5: 46..."
15657,AF-X7YGR2-F1-model_v4.pdb,90.442215,"{1: 35.88, 2: 37.38, 3: 39.41, 4: 41.41, 5: 42..."


In [38]:
df_af2_qual = pd.merge(df_plddt, data, on="model", how="inner")
df_af2_qual

,model,mean_plddt,resid_plddt,entryId,gene,uniprotAccession,uniprotId,uniprotDescription,taxId,organismScientificName,...,modelCreatedDate,latestVersion,allVersions,isReviewed,isReferenceProteome,cifUrl,bcifUrl,pdbUrl,paeImageUrl,paeDocUrl
0,AF-A0A009H9I3-F1-model_v4.pdb,94.549060,"{1: 43.47, 2: 38.41, 3: 41.44, 4: 45.84, 5: 43...",AF-A0A009H9I3-F1,ampC,A0A009H9I3,A0A009H9I3_9GAMM,Beta-lactamase,1310608,Acinetobacter sp. 1295259,...,2022-06-01,4,"[3, 4]",False,True,https://alphafold.ebi.ac.uk/files/AF-A0A009H9I...,https://alphafold.ebi.ac.uk/files/AF-A0A009H9I...,https://alphafold.ebi.ac.uk/files/AF-A0A009H9I...,https://alphafold.ebi.ac.uk/files/AF-A0A009H9I...,https://alphafold.ebi.ac.uk/files/AF-A0A009H9I...
1,AF-A0A009HR34-F1-model_v4.pdb,92.132418,"{1: 42.47, 2: 38.53000000000001, 3: 42.97, 4: ...",AF-A0A009HR34-F1,blaOXA-33,A0A009HR34,A0A009HR34_9GAMM,Beta-lactamase,1310608,Acinetobacter sp. 1295259,...,2022-06-01,4,"[3, 4]",False,True,https://alphafold.ebi.ac.uk/files/AF-A0A009HR3...,https://alphafold.ebi.ac.uk/files/AF-A0A009HR3...,https://alphafold.ebi.ac.uk/files/AF-A0A009HR3...,https://alphafold.ebi.ac.uk/files/AF-A0A009HR3...,https://alphafold.ebi.ac.uk/files/AF-A0A009HR3...
2,AF-A0A009I140-F1-model_v4.pdb,91.903540,"{1: 37.12, 2: 35.59, 3: 41.62, 4: 42.72, 5: 45...",AF-A0A009I140-F1,blaOXA-33,A0A009I140,A0A009I140_ACIB9,Beta-lactamase,1310613,Acinetobacter baumannii (strain 1295743),...,2022-06-01,4,"[3, 4]",False,True,https://alphafold.ebi.ac.uk/files/AF-A0A009I14...,https://alphafold.ebi.ac.uk/files/AF-A0A009I14...,https://alphafold.ebi.ac.uk/files/AF-A0A009I14...,https://alphafold.ebi.ac.uk/files/AF-A0A009I14...,https://alphafold.ebi.ac.uk/files/AF-A0A009I14...
3,AF-A0A009I9X2-F1-model_v4.pdb,94.540470,"{1: 41.28, 2: 37.47, 3: 39.28, 4: 43.56, 5: 45...",AF-A0A009I9X2-F1,blaCMY-10,A0A009I9X2,A0A009I9X2_ACIB9,Beta-lactamase,1310613,Acinetobacter baumannii (strain 1295743),...,2022-06-01,4,"[3, 4]",False,True,https://alphafold.ebi.ac.uk/files/AF-A0A009I9X...,https://alphafold.ebi.ac.uk/files/AF-A0A009I9X...,https://alphafold.ebi.ac.uk/files/AF-A0A009I9X...,https://alphafold.ebi.ac.uk/files/AF-A0A009I9X...,https://alphafold.ebi.ac.uk/files/AF-A0A009I9X...
4,AF-A0A009MI74-F1-model_v4.pdb,94.668590,"{1: 43.53, 2: 38.19, 3: 41.25, 4: 46.75, 5: 45...",AF-A0A009MI74-F1,J551_2290,A0A009MI74,A0A009MI74_9GAMM,Beta-lactamase,1310652,Acinetobacter sp. 1475718,...,2022-06-01,4,"[3, 4]",False,True,https://alphafold.ebi.ac.uk/files/AF-A0A009MI7...,https://alphafold.ebi.ac.uk/files/AF-A0A009MI7...,https://alphafold.ebi.ac.uk/files/AF-A0A009MI7...,https://alphafold.ebi.ac.uk/files/AF-A0A009MI7...,https://alphafold.ebi.ac.uk/files/AF-A0A009MI7...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15655,AF-X6KLF7-F1-model_v4.pdb,90.849410,"{1: 42.19, 2: 42.38, 3: 44.81, 4: 52.56, 5: 50...",AF-X6KLF7-F1,X726_06200,X6KLF7,X6KLF7_9HYPH,Beta-lactamase,1287074,Mesorhizobium sp. L103C105A0,...,2022-06-01,4,"[3, 4]",False,True,https://alphafold.ebi.ac.uk/files/AF-X6KLF7-F1...,https://alphafold.ebi.ac.uk/files/AF-X6KLF7-F1...,https://alphafold.ebi.ac.uk/files/AF-X6KLF7-F1...,https://alphafold.ebi.ac.uk/files/AF-X6KLF7-F1...,https://alphafold.ebi.ac.uk/files/AF-X6KLF7-F1...
15656,AF-X7FDK5-F1-model_v4.pdb,93.604694,"{1: 37.16, 2: 39.88, 3: 39.34, 4: 41.25, 5: 42...",AF-X7FDK5-F1,RISW2_18170,X7FDK5,X7FDK5_9RHOB,Beta-lactamase,1449351,Roseivivax isoporae LMG 25204,...,2022-06-01,4,"[3, 4]",False,True,https://alphafold.ebi.ac.uk/files/AF-X7FDK5-F1...,https://alphafold.ebi.ac.uk/files/AF-X7FDK5-F1...,https://alphafold.ebi.ac.uk/files/AF-X7FDK5-F1...,https://alphafold.ebi.ac.uk/files/AF-X7FDK5-F1...,https://alphafold.ebi.ac.uk/files/AF-X7FDK5-F1...
15657,AF-X7R747-F1-model_v4.pdb,95.022297,"{1: 39.09, 2: 37.66, 3: 42.78, 4: 44.12, 5: 46...",AF-X7R747-F1,Unknown,X7R747,X7R747_VIBPH,Beta-lactamase,1449358,Vibrio parahaemolyticus M0605,...,2022-06-01,4,"[3, 4]",False,False,https://alphafold.ebi.ac.uk/files/AF-X7R

In [39]:
df_af2_qual = df_af2_qual.sort_values(by='mean_plddt')
df_af2_qual = df_af2_qual.drop_duplicates(subset='uniprotSequence', keep='last')
df_af2_qual['dup_entry'] = df_af2_qual.groupby('uniprotSequence')['uniprotAccession'].transform(lambda x: list(x))

df_af2_qual

,model,mean_plddt,resid_plddt,entryId,gene,uniprotAccession,uniprotId,uniprotDescription,taxId,organismScientificName,...,latestVersion,allVersions,isReviewed,isReferenceProteome,cifUrl,bcifUrl,pdbUrl,paeImageUrl,paeDocUrl,dup_entry
10908,AF-A0A7M3SAR5-F1-model_v4.pdb,66.008026,"{1: 32.25, 2: 33.0, 3: 39.31, 4: 41.84, 5: 40....",AF-A0A7M3SAR5-F1,bsdcttw_47230,A0A7M3SAR5,A0A7M3SAR5_9FIRM,Transpeptidase domain-containing protein,2736602,Anaerocolumna sp. CTTW,...,4,"[3, 4]",False,True,https://alphafold.ebi.ac.uk/files/AF-A0A7M3SAR...,https://alphafold.ebi.ac.uk/files/AF-A0A7M3SAR...,https://alphafold.ebi.ac.uk/files/AF-A0A7M3SAR...,https://alphafold.ebi.ac.uk/files/AF-A0A7M3SAR...,https://alphafold.ebi.ac.uk/files/AF-A0A7M3SAR...,A0A7M3SAR5
4225,AF-A0A1H4IA45-F1-model_v4.pdb,69.834696,"{1: 63.09, 2: 72.25, 3: 79.06, 4: 87.81, 5: 87...",AF-A0A1H4IA45-F1,SAMN04490356_0289,A0A1H4IA45,A0A1H4IA45_STRMJ,Beta-lactamase,67327,Streptomyces melanosporofaciens,...,4,"[3, 4]",False,True,https://alphafold.ebi.ac.uk/files/AF-A0A1H4IA4...,https://alphafold.ebi.ac.uk/files/AF-A0A1H4IA4...,https://alphafold.ebi.ac.uk/files/AF-A0A1H4IA4...,https://alphafold.ebi.ac.uk/files/AF-A0A1H4IA4...,https://alphafold.ebi.ac.uk/files/AF-A0A1H4IA4...,A0A1H4IA45
15400,AF-V6H954-F1-model_v4.pdb,72.988365,"{1: 38.84, 2: 34.97, 3: 37.88, 4: 36.75, 5: 39...",AF-V6H954-F1,LEP1GSC047_1860,V6H954,V6H954_9LEPT,"Penicillin-binding protein, transpeptidase dom...",1049790,Leptospira inadai serovar Lyme str. 10,...,4,"[3, 4]",False,True,https://alphafold.ebi.ac.uk/files/AF-V6H954-F1...,https://alphafold.ebi.ac.uk/files/AF-V6H954-F1...,https://alphafold.ebi.ac.uk/files/AF-V6H954-F1...,https://alphafold.ebi.ac.uk/files/AF-V6H954-F1...,https://alphafold.ebi.ac.uk/files/AF-V6H954-F1...,V6H954
12891,AF-F5J1N6-F1-model_v4.pdb,73.207673,"{1: 31.19, 2: 27.33, 3: 29.34, 4: 27.61, 5: 30...",AF-F5J1N6-F1,HMPREF9455_03253,F5J1N6,F5J1N6_9BACT,Beta-lactamase,742766,Dysgonomonas gadei ATCC BAA-286,...,4,"[3, 4]",False,True,https://alphafold.ebi.ac.uk/files/AF-F5J1N6-F1...,https://alphafold.ebi.ac.uk/files/AF-F5J1N6-F1...,https://alphafold.ebi.ac.uk/files/AF-F5J1N6-F1...,https://alphafold.ebi.ac.uk/files/AF-F5J1N6-F1...,https://alphafold.ebi.ac.uk/files/AF-F5J1N6-F1...,F5J1N6
9411,AF-A0A4V6NGM0-F1-model_v4.pdb,74.089761,"{1: 52.0, 2: 51.46999999999999, 3: 51.62, 4: 5...",AF-A0A4V6NGM0-F1,EDD76_111146,A0A4V6NGM0,A0A4V6NGM0_9FIRM,Beta-lactamase class D,1469948,Kineothrix alysoides,...,4,"[3, 4]",False,True,https://alphafold.ebi.ac.uk/files/AF-A0A4V6NGM...,https://alphafold.ebi.ac.uk/files/AF-A0A4V6NGM...,https://alphafold.ebi.ac.uk/files/AF-A0A4V6NGM...,https://alphafold.ebi.ac.uk/files/AF-A0A4V6NGM...,https://alphafold.ebi.ac.uk/files/AF-A0A4V6NGM...,A0A4V6NGM0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2652,AF-A0A127SCJ5-F1-model_v4.pdb,98.430436,"{1: 83.06000000000002, 2: 97.56, 3: 98.12, 4: ...",AF-A0A127SCJ5-F1,Unknown,A0A127SCJ5,A0A127SCJ5_9BACT,Beta-lactamase,77133,uncultured bacterium,...,4,"[3, 4]",False,False,https://alphafold.ebi.ac.uk/files/AF-A0A127SCJ...,https://alphafold.ebi.ac.uk/files/AF-A0A127SCJ...,https://alphafold.ebi.ac.uk/files/AF-A0A127SCJ...,https://alphafold.ebi.ac.uk/files/AF-A0A127SCJ...,https://alphafold.ebi.ac.uk/files/AF-A0A127SCJ...,A0A127SCJ5
13083,AF-G8HDC7-F1-model_v4.pdb,98.442700,"{1: 78.19, 2: 94.75, 3: 95.75, 4: 97.5, 5: 97....",AF-G8HDC7-F1,bla,G8HDC7,G8HDC7_9BACT,Beta-lactamase,164851,uncultured soil bacterium,...,4,"[3, 4]",False,False,https://alphafold.ebi.ac.uk/files/AF-G8HDC7-F1...,https://alphafold.ebi.ac.uk/files/AF-G8HDC7-F1...,https://alphafold.ebi.ac.uk/files/AF-G8HDC7-F1...,https://alphafold.ebi.ac.uk/files/AF-G8HDC7-F1...,https://alphafold.ebi.ac.uk/files/AF-G8HDC7-F1...,G8HDC7
13062,AF-G8A3S3-F1-model_v4.pdb,98.467489,"{1: 75.12, 2: 94.5, 3: 98.19, 4: 98.19, 5: 98....",AF-G8A3S3-F1,blaTEM,G8A3S3,G8A3S3_PROVU,Beta-lactamase,585,Proteus vulgaris,...,4,"[3, 4]",False,False,https://alphafold.ebi.a

In [40]:
# df_plddt['resid_plddt'] = df_plddt['resid_plddt'].astype(str)
df_af2_qual.to_parquet("../bldb/af2/plddt_values_nonredu.parquet", index=False)